# Proyek Analisis Data: Bike Sharing
- **Nama:** Farell Kurniawan
- **Email:** farellkurniawan17108@gmail.com
- **ID Dicoding:** farellkurniawan17

## Menentukan Pertanyaan Bisnis

- **Pertanyaan 1:** Bagaimana tren penggunaan sepeda harian selama periode tertentu dan apakah faktor cuaca serta hari kerja/holiday berpengaruh signifikan terhadapnya?
- **Pertanyaan 2:** Bagaimana pola penggunaan sepeda berdasarkan jam dalam sehari, dan pada jam berapa terdapat puncak penggunaan?

## Import Semua Packages/Library yang Digunakan

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st
from datetime import datetime

## Data Wrangling

### Gathering Data

In [ ]:
# Membaca dataset
day_df = pd.read_csv('data/day.csv')
hour_df = pd.read_csv('data/hour.csv')

# Menampilkan 5 baris pertama untuk masing-masing dataset
print("5 Baris Pertama day_df:")
print(day_df.head())
print("\n5 Baris Pertama hour_df:")
print(hour_df.head())

**Insight:**
- Dataset **day.csv** memberikan informasi agregat harian.
- Dataset **hour.csv** menyediakan data granular per jam.
- Kedua dataset memiliki fitur seperti tanggal, kondisi cuaca, dan indikator hari libur/kerja.

### Assessing Data

In [ ]:
print("\nInformasi dan Ringkasan Statistik untuk day_df:")
print(day_df.info())
print(day_df.describe())

print("\nInformasi dan Ringkasan Statistik untuk hour_df:")
print(hour_df.info())
print(hour_df.describe())

**Insight:**
- Data memiliki tipe data yang sesuai dan tidak terdapat banyak missing values.
- Statistik deskriptif memberikan gambaran sebaran data yang bervariasi, sehingga terdapat potensi insight mendalam untuk analisis lebih lanjut.


### Cleaning Data

In [ ]:
# Mengonversi kolom tanggal menjadi datetime
day_df['dteday'] = pd.to_datetime(day_df['dteday'])
hour_df['dteday'] = pd.to_datetime(hour_df['dteday'])

# Cek missing values
print("\nCek Missing Values pada day_df:")
print(day_df.isnull().sum())
print("\nCek Missing Values pada hour_df:")
print(hour_df.isnull().sum())

**Insight:**
- Kolom `dteday` berhasil dikonversi ke format datetime, memudahkan analisis time series.
- Tidak ada missing values yang signifikan, sehingga dataset siap untuk analisis selanjutnya.

## Exploratory Data Analysis (EDA)

### Explore ...

In [ ]:
# Visualisasi tren penyewaan sepeda harian
plt.figure(figsize=(12, 6))
sns.lineplot(data=day_df, x='dteday', y='cnt', color='steelblue')
plt.title('Tren Penyewaan Sepeda Harian')
plt.xlabel('Tanggal')
plt.ylabel('Jumlah Penyewaan')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

**Insight:**
- Visualisasi tren harian menunjukkan fluktuasi penyewaan sepeda.
- Terlihat adanya pola musiman dan kemungkinan adanya pengaruh faktor eksternal seperti cuaca atau hari libur.

## Visualization & Explanatory Analysis

### Pertanyaan 1:
Bagaimana tren penggunaan sepeda harian selama periode tertentu dan apakah faktor cuaca serta hari kerja/holiday berpengaruh signifikan terhadapnya?

In [ ]:
# Visualisasi 1: Tren Harian & Pengaruh Hari Kerja vs Libur
plt.figure(figsize=(12, 6))
sns.lineplot(data=day_df, x='dteday', y='cnt', hue='workingday', palette='Set2')
plt.title('Tren Penyewaan Sepeda Harian: Hari Kerja vs Libur')
plt.xlabel('Tanggal')
plt.ylabel('Jumlah Penyewaan')
plt.legend(title="Hari Kerja (1) vs Libur (0)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Visualisasi 2: Penyewaan Sepeda Berdasarkan Kondisi Cuaca
plt.figure(figsize=(10, 6))
sns.boxplot(x='weathersit', y='cnt', data=day_df, palette='pastel')
plt.title('Penyewaan Sepeda Berdasarkan Kondisi Cuaca')
plt.xlabel('Kondisi Cuaca (1: Clear, 2: Misty, 3: Light Rain, 4: Heavy Rain)')
plt.ylabel('Jumlah Penyewaan')
plt.tight_layout()
plt.show()

**Insight:**
- Tren harian menunjukkan perbedaan signifikan antara hari kerja dan hari libur.
- Penyewaan cenderung lebih tinggi pada kondisi cuaca yang lebih baik (misalnya, weathersit=1) dan lebih rendah pada kondisi cuaca ekstrem.
- Hal ini mengindikasikan bahwa manajemen operasional dapat mempertimbangkan cuaca dan kalender kerja/libur untuk pengaturan armada.

### Pertanyaan 2:
Bagaimana pola penggunaan sepeda berdasarkan jam dalam sehari, dan pada jam berapa terdapat puncak penggunaan?

In [ ]:
# Menghitung rata-rata penyewaan per jam
hourly_avg = hour_df.groupby('hr')['cnt'].mean().reset_index()

# Visualisasi 1: Rata-Rata Penyewaan Sepeda per Jam (Line Plot)
plt.figure(figsize=(12, 6))
sns.lineplot(data=hourly_avg, x='hr', y='cnt', marker='o', color='coral')
plt.title('Rata-Rata Penyewaan Sepeda per Jam')
plt.xlabel('Jam')
plt.ylabel('Rata-Rata Penyewaan')
plt.xticks(range(0, 24))
plt.tight_layout()
plt.show()

# Visualisasi 2: Total Penyewaan Sepeda per Jam (Bar Plot)
plt.figure(figsize=(12, 6))
sns.barplot(x='hr', y='cnt', data=hour_df, estimator=sum, ci=None, color='mediumseagreen')
plt.title('Total Penyewaan Sepeda per Jam')
plt.xlabel('Jam')
plt.ylabel('Total Penyewaan')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

**Insight:**
- Dari visualisasi line plot, dapat dilihat jam-jam dengan rata-rata penyewaan tertinggi, yang kemungkinan merupakan jam sibuk (misalnya, pagi dan sore hari).
- Bar plot total penyewaan per jam memberikan gambaran volume penyewaan dalam angka nyata.
- Informasi ini penting untuk pengaturan armada, penjadwalan pemeliharaan, dan optimasi layanan.

## Analisis Lanjutan (Opsional)

Pengelompokkan Hari Berdasarkan Volume Penyewaan

(Tanpa menggunakan algoritma machine learning kompleks, cukup segmentasi sederhana)

In [ ]:
day_df['day_of_week'] = day_df['dteday'].dt.day_name()
avg_by_day = day_df.groupby('day_of_week')['cnt'].mean().reset_index()

plt.figure(figsize=(10, 6))
sns.barplot(x='day_of_week', y='cnt', data=avg_by_day, palette='viridis')
plt.title('Rata-Rata Penyewaan Sepeda Berdasarkan Hari dalam Seminggu')
plt.xlabel('Hari dalam Seminggu')
plt.ylabel('Rata-Rata Penyewaan')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

**Insight:** Analisis tambahan ini membantu untuk melihat apakah terdapat perbedaan signifikan dalam penyewaan antar hari (misalnya, weekend vs weekday).


## Conclusion

1.  Conclution **Pertanyaan 1**: Bagaimana tren penggunaan sepeda harian selama periode tertentu dan apakah faktor cuaca serta hari kerja/holiday berpengaruh signifikan terhadapnya?
  - Tren harian menunjukkan adanya fluktuasi penyewaan sepeda yang signifikan.
  - Data mengindikasikan bahwa hari kerja dan libur memiliki pola yang berbeda, di mana penyewaan cenderung lebih tinggi pada hari kerja dan saat kondisi cuaca baik.
  - Informasi ini dapat membantu dalam pengaturan armada dan penjadwalan operasional yang optimal.

2. Conclution **Pertanyaan 2**: Bagaimana pola penggunaan sepeda berdasarkan jam dalam sehari, dan pada jam berapa terdapat puncak penggunaan?
    - Analisis per jam mengungkapkan adanya puncak penggunaan pada waktu tertentu, misalnya pada pagi dan sore hari, yang mungkin berkaitan dengan jam berangkat dan pulang kerja.
    - Dengan mengetahui jam puncak, perusahaan dapat menyesuaikan distribusi sepeda dan meningkatkan efisiensi layanan, termasuk dalam penjadwalan pemeliharaan.

## Penutup

Dokumentasi lengkap, insight di setiap tahapan, serta visualisasi yang jelas diharapkan dapat memberikan gambaran yang komprehensif atas analisis data Bike Sharing. Selanjutnya, dashboard sederhana telah dibuat menggunakan Streamlit (lihat file dashboard.py) dan sudah dideploy pada Streamlit Cloud (tautan dashboard terdapat di file url.txt).